# Group: Synergy

This code is used for data processing for visualization parts

In [ ]:
import sqlite3
import json
from pprint import pprint
import pandas as pd
import numpy as np
import re

In [3]:
#Open the Json file
testFile = open("result_full.json")

In [4]:
data = json.load(testFile)
df = pd.DataFrame(data)

In [6]:
df = df.transpose()

# get rid of 'None'
df = df[df.Name.notnull()]

# edit column names
df.columns = ['Reviews Count', 'Introduction', 'Name', 'Ratings', 'Stars', 'Reviews',
              'Author', 'Category', 'Language', 'Pages', 'Price', 'Publisher', 'Rank', 'Tags']

In [7]:
# add ISBN list to the existing columns and change row index name
df['ISBN'] = pd.Series(df.index.tolist(),index = df.index)
df.index = range(17406)

In [9]:
#rearrange columns
df1 = df[['ISBN','Name','Author','Language','Category','Tags','Pages','Price','Publisher','Rank','Introduction','Reviews Count','Ratings','Stars','Reviews']]

In [10]:
df1.loc[:,'Reviews Count'] = df1.loc[:,'Reviews Count'].apply(lambda x: str(x)[:-16].strip())
df1.loc[:,'Pages'] = df1.loc[:,'Pages'].apply(lambda x: str(x)[:-6])

In [11]:
#language
array = df1.Language.unique()

In [12]:
for i in range(len(array)):
    array[i] = array[i].split()

In [13]:
array_flattened = [y for x in array for y in x]

In [14]:
language_list = sorted(set(array_flattened))
language_list.remove('English,')
language_list.remove('French,')
language_list.remove('German,')

In [17]:
# dummy variables for languages:
for language in language_list:
    list1 = df1.Language.tolist()
    for i in range(len(list1)):
        if language in list1[i]:
            list1[i] = 1
        else:
            list1[i] = 0
    df1[language] = pd.Series(list1, index=df1.index)

In [18]:
#language distribution:
for language in language_list:
    print("%7d\t%-10s"%(df1[language].sum(), language))

      1	Brazilian 
      1	Catalan   
      4	Chinese   
      2	Dutch     
  16978	English   
    104	French    
     86	German    
      1	Greek     
     13	Italian   
      2	Japanese  
      1	Latin     
      1	Multilingual
      1	Old       
      4	Portuguese
      1	Serbian   
    251	Spanish   
      3	Taiwanese 


In [19]:
# dummy variables for categories
category_array = df1.Category.unique()
for category in category_array:
    list1 = df1.Category.tolist()
    for i in range(len(list1)):
        if category in list1[i]:
            list1[i] = 1
        else:
            list1[i] = 0
    df1[category] = pd.Series(list1, index=df1.index)

In [20]:
#category distribution:
for category in category_array:
    print("%7d\t%-10s"%(df1[category].sum(),category))

     67	Engineering & Transportation
   4894	Literature & Fiction
   1799	Children's
    412	History   
    278	Science & Math
   1203	Mystery, Thriller & Suspense
    180	Cookbooks, Food & Wine
    577	Religion & Spirituality
     94	Medical   
    621	Politics & Social Sciences
    234	Business & Money
    822	Science Fiction & Fantasy
    332	Reference 
    561	Biographies & Memoirs
    220	Self-Help 
    133	Sports & Outdoors
    525	Teens     
    118	Parenting & Relationships
    598	Textbooks 
   1573	Romance   
    188	Travel    
    350	Humor & Entertainment
    342	Health, Fitness & Dieting
    206	Crafts, Hobbies & Home
     33	Law       
    220	Christian  & Bibles
     99	Comics & Graphic Novels
    134	Libros en espa ol
    293	Arts & Photography
    153	Computers & Technology
     17	Gay & Lesbian
     84	Education & Teaching
     16	on CD     
      1	Outdoor D cor
     28	Deals     
      1	Novelty & More


In [21]:
# extract publisher info from the 'Publisher' column
df1['Publisher_only'] = df1['Publisher']
df1.loc[:, 'Publisher_only'] = df1.loc[:, 'Publisher_only'].apply(
    lambda x: re.sub("[\(\[].*?[\)\]]", "", x))
df1.loc[:, 'Publisher_only'] = df1.loc[
    :, 'Publisher_only'].apply(lambda x: x.split(";", 1)[0])

In [22]:
pub_array = df1.Publisher_only.unique()

4417

In [23]:
#extract publish time info from the 'Publisher' column
regex = re.compile(".*?\((.*?)\)")
re.findall(regex, df1.Publisher[0])[0]

'September 3, 1979'

In [24]:
time_list = list()
for i in range(len(df1.Publisher)):
    result = re.findall(regex, df1.Publisher[i])
    time_list.append(result)

In [3]:
from dateutil import parser as dateparser
formatted_time = list()
for i in range(len(time_list)):
    try:
        formatted_time.append(dateparser.parse(
            ''.join(time_list[i])).strftime('%m/%Y'))
    except:
        formatted_time.append(None)

NameError: name 'time_list' is not defined

In [31]:
df1['Publish_time'] = pd.Series(formatted_time,index = df1.index)

In [26]:
ratings_dicts = df1.Ratings.tolist()

In [28]:
average_score = list()
for i in ratings_dicts:
    average_score.append(i.get('average score'))    

In [29]:
df1['Average_Rating'] = pd.Series(average_score,index = df1.index)

In [30]:
one_star_percent = list()
for i in ratings_dicts:
    one_star_percent.append(i.get('1'))
#one_star_percent

In [31]:
one_star_percent = [int(re.sub("%","",i)) for i in one_star_percent]
one_star_percent = [i/100 for i in one_star_percent]
#one_star_percent
df1['One_star'] = pd.Series(one_star_percent,index = df1.index)

In [32]:
two_star_percent = list()
for i in ratings_dicts:
    two_star_percent.append(i.get('2'))
two_star_percent = [int(re.sub("%","",i)) for i in two_star_percent]
two_star_percent = [i/100 for i in two_star_percent]
df1['Two_star'] = pd.Series(two_star_percent,index = df1.index)

In [33]:
three_star_percent = list()
for i in ratings_dicts:
    three_star_percent.append(i.get('3'))
three_star_percent = [int(re.sub("%","",i)) for i in three_star_percent]
three_star_percent = [i/100 for i in three_star_percent]
df1['Three_star'] = pd.Series(three_star_percent,index = df1.index)

In [34]:
four_star_percent = list()
for i in ratings_dicts:
    four_star_percent.append(i.get('4'))
four_star_percent = [int(re.sub("%","",i)) for i in four_star_percent]
four_star_percent = [i/100 for i in four_star_percent]
df1['Four_star'] = pd.Series(four_star_percent,index = df1.index)

In [35]:
five_star_percent = list()
for i in ratings_dicts:
    five_star_percent.append(i.get('5'))
five_star_percent = [int(re.sub("%","",i)) for i in five_star_percent]
five_star_percent = [i/100 for i in five_star_percent]
df1['Five_star'] = pd.Series(five_star_percent,index = df1.index)

In [36]:
#df1

In [37]:
#sales rank 
rank_list = df1.Rank.tolist()
rank_list = [int(re.sub(",","",i)) for i in rank_list]

In [2]:
df1['Rank'] = pd.Series(rank_list,index = df1.index)
# df1

NameError: name 'pd' is not defined

In [39]:
# Sentiment Analysis following the class example
p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'

In [40]:
import requests
words = requests.get(p_url).content.decode('latin-1')
# print(words)

In [41]:
# codes from classnotes
def get_words(url):
    import requests
    words = requests.get(url).content.decode('latin-1')

    word_list = words.split('\n')
    index = 0
    # Loop through the words/lines and remove what's not a word
    while index < len(word_list):
        word = word_list[index]
        if ';' in word or not word:
            word_list.pop(index)
        else:
            index += 1
    return word_list

# Get lists of positive and negative words
positive_words = get_words(p_url)
negative_words = get_words(n_url)


def remove_punctuation(word):
    if word and ((word[-1] >= 'a' and word[-1] <= 'z') or (word[-1] >= 'A' and word[-1] <= 'Z')):
        return word
    elif word:
        return word[:-1]
    else:
        return word

In [42]:
import nltk
from nltk.tokenize import sent_tokenize

In [43]:
# put all the reviews into bags of words to analyze positive/negative ratio
reviews_grand_list = df1.Reviews.tolist()
# reviews_grand_list
reviews_bag_of_words = list()
for i in reviews_grand_list:
    bag_of_words = list()
    for j in i.keys():
        bag_of_words.append(i.get(j))
    reviews_bag_of_words.append(bag_of_words)

In [44]:
# total reviews of all the books
total_reviews_bag = list()
for i in range(len(reviews_bag_of_words)):
    single_reviews_bag = list()
    for j in range(len(reviews_bag_of_words[i])):
        words_list = re.sub("[^\w]", " ",  reviews_bag_of_words[i][j]).split()
        single_reviews_bag.append(words_list)
    single_reviews_bag = [x for y in single_reviews_bag for x in y]
    total_reviews_bag.append(single_reviews_bag)

In [45]:
df1['Reviews_bag'] = pd.Series(total_reviews_bag,index = df1.index)

# END